In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
import kerastuner as kt

Using TensorFlow backend


C:\Users\jainh\AppData\Local\Temp\ipykernel_13820\134653722.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
df=pd.read_csv('diabetes2.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [5]:
st=StandardScaler()
X=st.fit_transform(X)

In [6]:
X.shape

(768, 8)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

In [8]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [9]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.6941 - accuracy: 0.6091 - val_loss: 0.6340 - val_accuracy: 0.6623
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6415 - accuracy: 0.6564 - val_loss: 0.5915 - val_accuracy: 0.7143
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6048 - accuracy: 0.6857 - val_loss: 0.5629 - val_accuracy: 0.7143
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5773 - accuracy: 0.7003 - val_loss: 0.5390 - val_accuracy: 0.7597
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5558 - accuracy: 0.7199 - val_loss: 0.5202 - val_accuracy: 0.8182
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5385 - accuracy: 0.7378 - val_loss: 0.5062 - val_accuracy: 0.8247
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5249 - accuracy: 0.7427 - val_loss: 0.4934 - val_accuracy: 0.8117
Epoch 8/100


Epoch 59/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4274 - accuracy: 0.7850 - val_loss: 0.4521 - val_accuracy: 0.7792
Epoch 60/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4265 - accuracy: 0.7818 - val_loss: 0.4530 - val_accuracy: 0.7792
Epoch 61/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4266 - accuracy: 0.7801 - val_loss: 0.4545 - val_accuracy: 0.7922
Epoch 62/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4258 - accuracy: 0.7883 - val_loss: 0.4553 - val_accuracy: 0.7857
Epoch 63/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4259 - accuracy: 0.7883 - val_loss: 0.4552 - val_accuracy: 0.7857
Epoch 64/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4256 - accuracy: 0.7899 - val_loss: 0.4537 - val_accuracy: 0.7857
Epoch 65/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4250 - accuracy: 0.7850 - val_loss: 0.4520 - val_accuracy: 0.7922
Epoch 

# 1.how to select appropriate optimizer
# 2.no. of neurons in one layer
# 3.how to select no. of layers
# 4.for number of layers and number of neurons in one layer, activation     
#    function
# 5. hyperparameter tunning for number of layers and number of neurons in one layer, activation function with drop out 

# 1.how to select appropriate optimizer

In [10]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [11]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir',project_name='optimizer_tunning')

In [12]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 07s


In [13]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [14]:
model=tuner.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 15ms/step - loss: 0.5240 - accuracy: 0.7394 - val_loss: 0.5211 - val_accuracy: 0.7662
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5098 - accuracy: 0.7459 - val_loss: 0.5101 - val_accuracy: 0.7597
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4980 - accuracy: 0.7590 - val_loss: 0.5010 - val_accuracy: 0.7597
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4888 - accuracy: 0.7687 - val_loss: 0.4943 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4822 - accuracy: 0.7752 - val_loss: 0.4899 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4765 - accuracy: 0.7687 - val_loss: 0.4866 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4718 - accuracy: 0.7704 - val_loss: 0.4833 - val_accuracy: 0.8052
Epoch 13/

Epoch 64/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4297 - accuracy: 0.7850 - val_loss: 0.4687 - val_accuracy: 0.7922
Epoch 65/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4290 - accuracy: 0.7785 - val_loss: 0.4685 - val_accuracy: 0.7922
Epoch 66/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4289 - accuracy: 0.7834 - val_loss: 0.4680 - val_accuracy: 0.7922
Epoch 67/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4285 - accuracy: 0.7818 - val_loss: 0.4677 - val_accuracy: 0.7922
Epoch 68/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4275 - accuracy: 0.7834 - val_loss: 0.4683 - val_accuracy: 0.7857
Epoch 69/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4277 - accuracy: 0.7818 - val_loss: 0.4680 - val_accuracy: 0.7987
Epoch 70/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4269 - accuracy: 0.7818 - val_loss: 0.4683 - val_accuracy: 0.7922
Epoch 

# 2.hyperparameter tunning for no. of neurons in one layer

In [17]:
df=pd.read_csv('diabetes2.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [19]:
st=StandardScaler()
X=st.fit_transform(X)

In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

In [21]:
def build_model(hp):
    model=Sequential()
    units=hp.Int('units',min_value=8,max_value=128,step=8)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [22]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir',project_name='no_neurons_tunning')

In [23]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 09s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [25]:
model=tuner.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 1s 11ms/step - loss: 0.5420 - accuracy: 0.7459 - val_loss: 0.5219 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5202 - accuracy: 0.7508 - val_loss: 0.5049 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5042 - accuracy: 0.7590 - val_loss: 0.4921 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4926 - accuracy: 0.7573 - val_loss: 0.4838 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4822 - accuracy: 0.7622 - val_loss: 0.4760 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4775 - accuracy: 0.7590 - val_loss: 0.4724 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4701 - accuracy: 0.7622 - val_loss: 0.4694 - val_accuracy: 0.7857
Epoch 13/

Epoch 64/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4226 - accuracy: 0.7883 - val_loss: 0.4647 - val_accuracy: 0.7922
Epoch 65/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4219 - accuracy: 0.7899 - val_loss: 0.4643 - val_accuracy: 0.7922
Epoch 66/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4215 - accuracy: 0.7899 - val_loss: 0.4632 - val_accuracy: 0.7987
Epoch 67/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4211 - accuracy: 0.7948 - val_loss: 0.4639 - val_accuracy: 0.7857
Epoch 68/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4214 - accuracy: 0.7948 - val_loss: 0.4656 - val_accuracy: 0.7857
Epoch 69/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4195 - accuracy: 0.7948 - val_loss: 0.4640 - val_accuracy: 0.7922
Epoch 70/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4207 - accuracy: 0.7834 - val_loss: 0.4617 - val_accuracy: 0.7922
Epoch 

# 3.how to select no. of layers

In [28]:
df=pd.read_csv('diabetes2.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [29]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [30]:
st=StandardScaler()
X=st.fit_transform(X)

In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

In [32]:
def build_model(hp):
    model=Sequential()
    model.add(Dense(72,activation='relu',input_dim=8))
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [33]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir',project_name='no_layers')

In [34]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 13s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [36]:
model=tuner.get_best_models(num_models=1)[0]

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 1)                 73        
                                                                 
Total params: 21745 (84.94 KB)
Trainable params: 21745 (

In [38]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 [==============================] - 2s 20ms/step - loss: 0.4366 - accuracy: 0.7866 - val_loss: 0.4670 - val_accuracy: 0.7987
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4248 - accuracy: 0.7964 - val_loss: 0.4637 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4117 - accuracy: 0.8029 - val_loss: 0.4626 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3956 - accuracy: 0.8046 - val_loss: 0.4711 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3891 - accuracy: 0.8208 - val_loss: 0.5055 - val_accuracy: 0.7857
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3950 - accuracy: 0.8094 - val_loss: 0.4765 - val_accuracy: 0.8182
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3757 - accuracy: 0.8208 - val_loss: 0.4830 - val_accuracy: 0.8052
Epoch 13/

Epoch 64/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1769 - accuracy: 0.9544 - val_loss: 1.7206 - val_accuracy: 0.7532
Epoch 65/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1117 - accuracy: 0.9560 - val_loss: 1.6056 - val_accuracy: 0.6948
Epoch 66/100
20/20 [==============================] - 0s 4ms/step - loss: 0.1084 - accuracy: 0.9495 - val_loss: 1.5996 - val_accuracy: 0.7338
Epoch 67/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0758 - accuracy: 0.9707 - val_loss: 1.4125 - val_accuracy: 0.7208
Epoch 68/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0376 - accuracy: 0.9902 - val_loss: 1.3897 - val_accuracy: 0.7468
Epoch 69/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0339 - accuracy: 0.9870 - val_loss: 1.5820 - val_accuracy: 0.7338
Epoch 70/100
20/20 [==============================] - 0s 3ms/step - loss: 0.0215 - accuracy: 0.9984 - val_loss: 1.6330 - val_accuracy: 0.7208
Epoch 

# 4. hyperparameter tunning for number of layers and number of neurons in one layer, activation function

In [39]:
df=pd.read_csv('diabetes2.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [40]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [41]:
st=StandardScaler()
X=st.fit_transform(X)

In [42]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

In [43]:
def build_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(
                Dense(
                    hp.Int('units' +str(i), min_value=8,max_value=128,step=4),
                            activation=hp.Choice('activation' +str(i), values=['tanh','relu','sigmoid']),
                            input_dim=8
                            )
                     )
        else:
            model.add(
                Dense(
                    hp.Int('units' +str(i), min_value=8,max_value=128,step=4),
                            activation=hp.Choice('activation' +str(i), values=['tanh','relu','sigmoid'])
                          )
                     )
        counter+=1
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta','nadam']),
                      loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [44]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir',project_name='all_hyp_tun')

In [45]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 14s


In [46]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 84,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 96,
 'activation1': 'tanh',
 'units2': 72,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'tanh',
 'units4': 36,
 'activation4': 'sigmoid',
 'units5': 88,
 'activation5': 'sigmoid',
 'units6': 40,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'tanh',
 'units8': 32,
 'activation8': 'sigmoid',
 'units9': 128,
 'activation9': 'relu'}

In [47]:
model=tuner.get_best_models(num_models=1)[0]

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 84)                756       
                                                                 
 dense_1 (Dense)             (None, 96)                8160      
                                                                 
 dense_2 (Dense)             (None, 1)                 97        
                                                                 
Total params: 9013 (35.21 KB)
Trainable params: 9013 (35.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
model.fit(X_train,y_train,batch_size=32,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 1s 12ms/step - loss: 0.4797 - accuracy: 0.7785 - val_loss: 0.4687 - val_accuracy: 0.7792
Epoch 7/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4741 - accuracy: 0.7622 - val_loss: 0.4690 - val_accuracy: 0.7792
Epoch 8/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4703 - accuracy: 0.7752 - val_loss: 0.4619 - val_accuracy: 0.7727
Epoch 9/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4729 - accuracy: 0.7736 - val_loss: 0.4712 - val_accuracy: 0.7727
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4617 - accuracy: 0.7785 - val_loss: 0.4650 - val_accuracy: 0.7792
Epoch 11/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4730 - accuracy: 0.7785 - val_loss: 0.4552 - val_accuracy: 0.8052
Epoch 12/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4666 - accuracy: 0.7769 - val_loss: 0.4722 - val_accuracy: 0.7792
Epoch 13/

Epoch 64/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4107 - accuracy: 0.8078 - val_loss: 0.4354 - val_accuracy: 0.7857
Epoch 65/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4062 - accuracy: 0.8257 - val_loss: 0.4482 - val_accuracy: 0.7922
Epoch 66/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4081 - accuracy: 0.8078 - val_loss: 0.4497 - val_accuracy: 0.7857
Epoch 67/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4069 - accuracy: 0.8127 - val_loss: 0.4367 - val_accuracy: 0.7922
Epoch 68/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4024 - accuracy: 0.8094 - val_loss: 0.4440 - val_accuracy: 0.7922
Epoch 69/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4025 - accuracy: 0.8160 - val_loss: 0.4437 - val_accuracy: 0.7987
Epoch 70/200
20/20 [==============================] - 0s 3ms/step - loss: 0.4057 - accuracy: 0.8127 - val_loss: 0.4457 - val_accuracy: 0.7857
Epoch 

20/20 [==============================] - 0s 3ms/step - loss: 0.3201 - accuracy: 0.8713 - val_loss: 0.4555 - val_accuracy: 0.8117
Epoch 122/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3169 - accuracy: 0.8730 - val_loss: 0.4697 - val_accuracy: 0.7987
Epoch 123/200
20/20 [==============================] - 0s 4ms/step - loss: 0.3186 - accuracy: 0.8697 - val_loss: 0.4624 - val_accuracy: 0.7922
Epoch 124/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3171 - accuracy: 0.8632 - val_loss: 0.4611 - val_accuracy: 0.8117
Epoch 125/200
20/20 [==============================] - 0s 3ms/step - loss: 0.3117 - accuracy: 0.8713 - val_loss: 0.4773 - val_accuracy: 0.7922
Epoch 126/200
20/20 [==============================] - 0s 5ms/step - loss: 0.3097 - accuracy: 0.8811 - val_loss: 0.4683 - val_accuracy: 0.8052
Epoch 127/200
20/20 [==============================] - 0s 6ms/step - loss: 0.3135 - accuracy: 0.8730 - val_loss: 0.4589 - val_accuracy: 0.8052
Epoch 128/200

20/20 [==============================] - 0s 3ms/step - loss: 0.2035 - accuracy: 0.9430 - val_loss: 0.5366 - val_accuracy: 0.7662
Epoch 179/200
20/20 [==============================] - 0s 3ms/step - loss: 0.2028 - accuracy: 0.9463 - val_loss: 0.5435 - val_accuracy: 0.7597
Epoch 180/200
20/20 [==============================] - 0s 6ms/step - loss: 0.2031 - accuracy: 0.9463 - val_loss: 0.5475 - val_accuracy: 0.7662
Epoch 181/200
20/20 [==============================] - 0s 6ms/step - loss: 0.1989 - accuracy: 0.9414 - val_loss: 0.5607 - val_accuracy: 0.7662
Epoch 182/200
20/20 [==============================] - 0s 4ms/step - loss: 0.1957 - accuracy: 0.9479 - val_loss: 0.5523 - val_accuracy: 0.7532
Epoch 183/200
20/20 [==============================] - 0s 4ms/step - loss: 0.1940 - accuracy: 0.9495 - val_loss: 0.5576 - val_accuracy: 0.7532
Epoch 184/200
20/20 [==============================] - 0s 4ms/step - loss: 0.1907 - accuracy: 0.9430 - val_loss: 0.5613 - val_accuracy: 0.7532
Epoch 185/200

In [50]:
# def build_model(hp):
#     model=Sequential()
#     counter=0
#     for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
#         if counter==0:
#             model.add(
#                 Dense(
#                     hp.Int('units' +str(i), min_value=8,max_value=128,step=4),
#                             activation=hp.Choice('activation' +str(i), values=['tanh','relu','sigmoid']),
#                             input_dim=8
#                             )
#                      )
#         else:
#             model.add(
#                 Dense(
#                     hp.Int('units' +str(i), min_value=8,max_value=128,step=4),
#                             activation=hp.Choice('activation' +str(i), values=['tanh','relu','sigmoid'])
#                           )
#                      )
#         counter+=1
#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(optimizer=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta','nadam']),
#                       loss='binary_crossentropy',metrics=['accuracy'])
#     return model

# 5. hyperparameter tunning for number of layers and number of neurons in one layer, activation function with drop out 

In [51]:
df=pd.read_csv('diabetes2.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [52]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [53]:
st=StandardScaler()
X=st.fit_transform(X)

In [54]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

In [55]:
def build_model(hp):
    model=Sequential()
    counter=0
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter==0:
            model.add(
                Dense(
                    hp.Int('units' +str(i), min_value=8,max_value=128,step=4),
                            activation=hp.Choice('activation' +str(i), values=['tanh','relu','sigmoid']),
                            input_dim=8
                            )
                     )
            model.add(
                Dropout(
                    hp.Choice('dropout' +str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
                       )
                     )
        else:
            model.add(
                Dense(
                    hp.Int('units' +str(i), min_value=8,max_value=128,step=4),
                            activation=hp.Choice('activation' +str(i), values=['tanh','relu','sigmoid'])
                          )
                     )
            model.add(
                Dropout(
                    hp.Choice('dropout' +str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
                       )
                     )
        counter+=1
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta','nadam']),
                      loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [56]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='my_dir',project_name='all_hyp_tun_with_dropout')

In [57]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6883116960525513
Total elapsed time: 00h 00m 12s


In [58]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 88,
 'activation0': 'sigmoid',
 'dropout0': 0.4,
 'optimizer': 'nadam',
 'units1': 76,
 'activation1': 'relu',
 'dropout1': 0.4,
 'units2': 64,
 'activation2': 'relu',
 'dropout2': 0.3,
 'units3': 84,
 'activation3': 'tanh',
 'dropout3': 0.3}

In [59]:
model=tuner.get_best_models(num_models=1)[0]

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 88)                792       
                                                                 
 dropout (Dropout)           (None, 88)                0         
                                                                 
 dense_1 (Dense)             (None, 76)                6764      
                                                                 
 dropout_1 (Dropout)         (None, 76)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 77        
                                                                 
Total params: 7633 (29.82 KB)
Trainable params: 7633 (29.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
model.fit(X_train,y_train,batch_size=32,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 2s 17ms/step - loss: 0.6014 - accuracy: 0.6710 - val_loss: 0.5423 - val_accuracy: 0.7273
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5919 - accuracy: 0.6889 - val_loss: 0.5311 - val_accuracy: 0.7597
Epoch 8/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5815 - accuracy: 0.6808 - val_loss: 0.5225 - val_accuracy: 0.7403
Epoch 9/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5726 - accuracy: 0.6889 - val_loss: 0.5099 - val_accuracy: 0.7792
Epoch 10/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5805 - accuracy: 0.7020 - val_loss: 0.4989 - val_accuracy: 0.7597
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5377 - accuracy: 0.7264 - val_loss: 0.4937 - val_accuracy: 0.7662
Epoch 12/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5487 - accuracy: 0.7329 - val_loss: 0.4941 - val_accuracy: 0.7662
Epoch 13/

Epoch 64/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4876 - accuracy: 0.7720 - val_loss: 0.4808 - val_accuracy: 0.7792
Epoch 65/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4928 - accuracy: 0.7524 - val_loss: 0.4842 - val_accuracy: 0.7792
Epoch 66/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5027 - accuracy: 0.7345 - val_loss: 0.4882 - val_accuracy: 0.7857
Epoch 67/200
20/20 [==============================] - 0s 4ms/step - loss: 0.5005 - accuracy: 0.7427 - val_loss: 0.4874 - val_accuracy: 0.7857
Epoch 68/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4889 - accuracy: 0.7655 - val_loss: 0.4844 - val_accuracy: 0.7727
Epoch 69/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4904 - accuracy: 0.7508 - val_loss: 0.4837 - val_accuracy: 0.7792
Epoch 70/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4835 - accuracy: 0.7638 - val_loss: 0.4852 - val_accuracy: 0.7857
Epoch 

20/20 [==============================] - 0s 5ms/step - loss: 0.4688 - accuracy: 0.7736 - val_loss: 0.4877 - val_accuracy: 0.7792
Epoch 122/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4804 - accuracy: 0.7752 - val_loss: 0.4876 - val_accuracy: 0.7792
Epoch 123/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4567 - accuracy: 0.7915 - val_loss: 0.4823 - val_accuracy: 0.7792
Epoch 124/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4665 - accuracy: 0.7818 - val_loss: 0.4813 - val_accuracy: 0.7792
Epoch 125/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4789 - accuracy: 0.7736 - val_loss: 0.4875 - val_accuracy: 0.7922
Epoch 126/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4772 - accuracy: 0.7687 - val_loss: 0.4885 - val_accuracy: 0.7792
Epoch 127/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4761 - accuracy: 0.7736 - val_loss: 0.4858 - val_accuracy: 0.7857
Epoch 128/200

20/20 [==============================] - 0s 4ms/step - loss: 0.4636 - accuracy: 0.7769 - val_loss: 0.4845 - val_accuracy: 0.7792
Epoch 179/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4674 - accuracy: 0.7834 - val_loss: 0.4818 - val_accuracy: 0.7857
Epoch 180/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4613 - accuracy: 0.7785 - val_loss: 0.4767 - val_accuracy: 0.7857
Epoch 181/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4719 - accuracy: 0.7671 - val_loss: 0.4818 - val_accuracy: 0.7857
Epoch 182/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4672 - accuracy: 0.7704 - val_loss: 0.4797 - val_accuracy: 0.7857
Epoch 183/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4690 - accuracy: 0.7866 - val_loss: 0.4807 - val_accuracy: 0.7857
Epoch 184/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4721 - accuracy: 0.7671 - val_loss: 0.4786 - val_accuracy: 0.7857
Epoch 185/200